# Decision trees

![img](https://pbs.twimg.com/media/B13n2VVCIAA0hJS.jpg)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Let's generate a toy dataset:

In [ ]:
from sklearn.datasets import make_blobs

X_toy, y_toy = make_blobs(n_samples=400,
                          centers=[[0., 1.], [1., 2.]],
                          random_state=14)

plt.scatter(X_toy[~(y_toy==1), 0], X_toy[~(y_toy==1), 1], alpha=0.8, marker='s', label='0', c='deepskyblue')
plt.scatter(X_toy[y_toy==1,    0], X_toy[y_toy==1,    1], alpha=0.8, marker='^', label='1', c='orange')

plt.legend()
plt.show()

## Decision trees out of the box

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import accuracy_score

DecisionTreeClassifier has a number of parameters:
* `max_depth` – a limit on tree depth (default – no limit)
* `min_samples_split` – there should be at least this many samples to split further (default – 2)
* `min_samples_leaf` – there should be at least this many samples on one side of a split to consider it valid (default – 1).
* `criterion` – 'gini' or 'entropy' – split stuff over this parameter (default : gini)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_toy, y_toy)

### Plot decision surface

You may also check [sklearn.inspection.DecisionBoundaryDisplay](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.DecisionBoundaryDisplay.html).

Let's plot the tree we've fitted above:

In [ ]:
plt.figure(figsize=(12, 8))
plot_decision_regions(X_toy, y_toy, clf);

### Tree depth

First we are going to split our data to train and test subsets:

In [ ]:
X_toy_train, X_toy_test, y_toy_train, y_toy_test = \
    train_test_split(X_toy, y_toy, test_size=0.25)

Let's investigate how the decision boundary depends on the tree depth. Maximum tree depth is defined by the `max_depth` parameter: depict   decision boundary plots for both train and test datasets (separately).


In [ ]:
depth_values = [1, 2, 3, 5, 10, None]

fig, axs = plt.subplots(ncols = 2, nrows = len(depth_values), figsize=(20, 30))


for i in range(len(depth_values)):
  max_depth = depth_values[i]

  dt = DecisionTreeClassifier(max_depth=max_depth, random_state=13)
  dt.fit(X_toy_train, y_toy_train)


  axs[i][0].set_title(
            "max_depth = {} Train accuracy = {} ".format(max_depth, accuracy_score(y_toy_train, dt.predict(X_toy_train)))
        )
  axs[i][0].axis("off")
  axs[i][1].set_title(
            "max_depth = {} Test  accuracy = {} ".format(max_depth, accuracy_score(y_toy_test, dt.predict(X_toy_test)))
        )
  axs[i][1].axis("off")
  plot_decision_regions(X_toy_train, y_toy_train, dt, ax=axs[i][0])
  plot_decision_regions(X_toy_test,  y_toy_test,  dt, ax=axs[i][1])

plt.show()


### Overfitting

Trees seem to overfit. Let's conduct an experiment: choose random 90\% of the sample and fit the tree. And check if they differ.

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(15, 12))

for i in range(3):
    for j in range(3):
        seed_idx = 3 * i + j
        np.random.seed(seed_idx)
        dt = DecisionTreeClassifier(random_state=13)
        idx_part = np.random.choice(len(X_toy_train), replace=False, size=int(0.9 * len(X_toy_train)))
        X_part, y_part = X_toy_train[idx_part, :], y_toy_train[idx_part]
        dt.fit(X_part, y_part)
        ax[i][j].set_title("sample #{}".format(seed_idx))
        ax[i][j].axis("off")
        plot_decision_regions(X_part, y_part, dt, ax=ax[i][j])

plt.show()

### Toy multiclass data

Now let's try out a multiclass classification case:

In [ ]:
!wget https://raw.githubusercontent.com/Majid-Sohrabi/MLDM-2025/main/07-trees/data.npz

Firstly, we'll load the data:

In [ ]:
data = np.load('data.npz')
X, y = data["X"], data["y"]

print(X.shape, y.shape)

And then split it to train and test:

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.5, random_state=1337)

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='rainbow', s = 10)

Now that we've had a look at the data, let's fit a decision tree on it:

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

In [ ]:
print('Train accuracy = ', accuracy_score(y_train, clf.predict(X_train)))
print('Test  accuracy = ', accuracy_score(y_test, clf.predict(X_test)))

and plot the result:

In [ ]:
plt.figure(figsize=(12, 16))
plt.subplot(2, 1, 1)
plot_decision_regions(X_train, y_train, clf)
plt.subplot(2, 1, 2)
plot_decision_regions(X_test, y_test, clf);

#### We need a better tree!

Try adjusting the parameters of DecisionTreeClassifier to improve the test accuracy.
 * Accuracy >= 0.72 - not bad for a start
 * Accuracy >= 0.75 - better, but not enough
 * Accuracy >= 0.77 - pretty good
 * Accuracy >= 0.78 - great! (probably the best result for a single tree) **(1 point)**

Feel free to modify the DecisionTreeClassifier above instead of re-writing everything.

**Note:** some of the parameters you can tune are under the "Decision trees out of the box" header.

## Feature transformations
Try adding feature transformations using a pipeline and a transformation, e.g. function transformer.

```python
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

clf = make_pipeline(
    FunctionTransformer(lambda X: np.concatenate([X, X**2], axis=1)),
    DecisionTreeClassifier()
)
```

Which transformations should improve the score?

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

clf = make_pipeline(
FunctionTransformer(lambda X: X),
DecisionTreeClassifier(random_state=42)
)

clf.fit(X_train, y_train)

print('Train accuracy = ', accuracy_score(y_train, clf.predict(X_train)))
print('Test  accuracy = ', accuracy_score(y_test, clf.predict(X_test)))



plt.figure(figsize=(12, 16))
plt.subplot(2, 1, 1)
plot_decision_regions(X_train, y_train, clf)
plt.subplot(2, 1, 2)
plot_decision_regions(X_test, y_test, clf)

```
```
```
```

We've talked a lot about the importance of feature scaling. Why aren't we doing it here?

Let's try adding a standard scaler to the pipeline of our model and check how it affects the result. Can you explain the result?

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
clf = DecisionTreeClassifier(max_depth = 3, min_samples_split=6)
clf.fit(X_train, y_train)

print('No scaling')
print('Train accuracy = ', accuracy_score(y_train, clf.predict(X_train)))
print('Test  accuracy = ', accuracy_score(y_test, clf.predict(X_test)))

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
clf.fit(X_train_scaled, y_train)

print('With scaling')
print('Train accuracy = ', accuracy_score(y_train, clf.predict(X_train_scaled)))
print('Test  accuracy = ', accuracy_score(y_test, clf.predict(X_test_scaled)))

## Visualizing a tree

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(15, 8), dpi=100)
plot_tree(clf, fontsize=10, filled=True);

## Tree pruning (Minimal Cost-Complexity)

Let tree $T$ have the total weighted sample impurity of the terminal nodes $R(T)$.

Can prune the tree by minimizing:
$$R_\alpha(T) = R(T) + \alpha\left|T\right|,$$
where $\alpha\geq0$, and $\left|T\right|$ is the number of terminal nodes in the tree.

Let $T_t$ be the subtree tree whose root node is $t\in T$.

$T_t$ will be pruned out (i.e. replaced with $t$ as the terminal node) if:
$$R(t)+\alpha < R(T_t)+\alpha\left|T_t\right|$$
or in other words if:
$$\alpha > \alpha_{eff}(t)=\frac{R(t) - R(T_t)}{\left|T_t\right|-1}$$

One can use the `cost_complexity_pruning_path` method of `DecisionTreeClassifier` to get the list of $\alpha_{eff}$.

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

path = clf.cost_complexity_pruning_path(X_train, y_train)

ccp_alphas, impurities = path.ccp_alphas, path.impurities

plt.plot(ccp_alphas, impurities, marker='o', drawstyle="steps-post")
plt.xlabel("effective alpha")
plt.ylabel("total impurity of leaves")
plt.title("Total Impurity vs effective alpha for training set");

 `DecisionTreeClassifier` has a `ccp_alpha` parameter to prune the tree.

 For each of the `ccp_alphas` defined above fit a tree, and then make 3 plots:
 - tree depth vs alpha
 - number of nodes in the tree vs alpha
 - train and test accuracies vs alpha

Attribute  `clf.tree_.max_depth` gives the tree depth , and `clf.tree_.node_count` - number of nodes.

In [ ]:
tree_depth = []
number_of_nodes = []
train_acc = []
test_acc = []
for alpha in ccp_alphas:
    clf = DecisionTreeClassifier(ccp_alpha = alpha).fit(X_train, y_train)

    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)

    train_acc.append(accuracy_score(y_train, y_train_pred))
    test_acc.append(accuracy_score(y_test, y_test_pred))

    number_of_nodes.append(clf.tree_.node_count)
    tree_depth.append(clf.tree_.max_depth)

In [ ]:
plt.figure(figsize=(5*3, 5))

plt.subplot(1, 3, 1)
plt.plot(ccp_alphas, train_acc, label='Train accuracy')
plt.plot(ccp_alphas, test_acc,  label='Test accuracy')
plt.title('Accuracy')
plt.xlabel(r'$\alpha$')
plt.legend()
plt.grid()


plt.subplot(1, 3, 2)
plt.plot(ccp_alphas, number_of_nodes)
plt.title('Number of nodes')
plt.xlabel(r'$\alpha$')
plt.grid()

plt.subplot(1, 3, 3)
plt.plot(ccp_alphas, tree_depth)
plt.title('Tree depth')
plt.xlabel(r'$\alpha$')
plt.grid()


In [ ]:
ccp_alphas[np.argmax(test_acc)]

## Decision trees vs linear models

This data is easy for linear models, but hard for trees.

In [ ]:
np.random.seed(13)
n = 500
X = np.zeros(shape=(n, 2))
X[:, 0] = np.linspace(-5, 5, 500)
X[:, 1] = X[:, 0] + 0.5 * np.random.normal(size=n)
y = (X[:, 1] > X[:, 0]).astype(int)
plt.scatter(X[:, 0], X[:, 1], s=10, c=y);

In [ ]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=13
)

lr = LogisticRegression(random_state=13)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print(f"Linear model accuracy: {accuracy_score(y_pred_lr, y_test):.2f}")

plot_decision_regions(X_test, y_test, lr);

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=13)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print(f"Decision tree accuracy: {accuracy_score(y_pred_dt, y_test):.2f}")

plot_decision_regions(X_test, y_test, dt);

## Decision tree for regression

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import  mean_squared_error

import pandas as pd

california = fetch_california_housing()
california_X = pd.DataFrame(data=california.data, columns=california.feature_names)
california_Y = california.target
print(f"X shape: {california_X.shape}, Y shape: {california_Y.shape}")
california_X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    california_X, california_Y, test_size=0.25, random_state=13
)

dt = DecisionTreeRegressor(max_depth=3, random_state=13)
dt.fit(X_train, y_train)

print(mean_squared_error(y_test, dt.predict(X_test)))

plt.figure(figsize=(10, 10))
plot_tree(dt, feature_names=california_X.columns, filled=True, rounded=True)
plt.show()

In [ ]:
max_depth_array = range(2, 20)
mse_array = []

for max_depth in max_depth_array:
    dt = DecisionTreeRegressor(max_depth=max_depth, random_state=13)
    dt.fit(X_train, y_train)
    mse_array.append(mean_squared_error(y_test, dt.predict(X_test)))

plt.plot(max_depth_array, mse_array)
plt.title("Dependence of MSE on max depth")
plt.xlabel("max depth")
plt.ylabel("MSE");

In [ ]:
max_depth_array[np.argmin(mse_array)]

Let's search for best hyperparameters using grid search:

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(DecisionTreeRegressor(random_state=13),
                  param_grid={
                      'max_features': ['auto', 'log2', 'sqrt'],
                      'max_depth': list(range(2, 20)) + [None],
                      'min_samples_leaf': list(range(20, 70, 3)) + [None]
                  },
                  cv=5,
                  scoring='neg_mean_squared_error',
                  verbose=5)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
print(mean_squared_error(y_test, gs.predict(X_test)))

## Feature importance

In [ ]:
dt = DecisionTreeRegressor(random_state=13, **gs.best_params_)
dt.fit(X_train, y_train)

df_importances = pd.DataFrame(
    {"feature": california_X.columns, "importance": dt.feature_importances_}
).sort_values(by="importance", ascending=False).reset_index(drop=True)

In [ ]:
plt.bar(df_importances['feature'], df_importances['importance'])
plt.xticks(rotation=20)
plt.show()

Linear dependency of features may influence the results:

In [ ]:
new_feature1 = np.array((X_train.iloc[:, 0] * 2 + 3)).reshape(-1, 1)
new_feature2 = np.array((X_train.iloc[:, 0] * 3 - 2)).reshape(-1, 1)
X_train_new = np.hstack((X_train, new_feature1, new_feature2))

dt.fit(X_train_new, y_train)

df_importances = pd.DataFrame(
    {"feature": np.concatenate([california_X.columns, ['MedInc1', 'MedInc2']]), "importance": dt.feature_importances_}
).sort_values(by="importance", ascending=False).reset_index(drop=True)

plt.bar(df_importances['feature'], df_importances['importance'])
plt.xticks(rotation=20)
plt.show()